In [390]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
import gradio as gr

In [391]:
games_df = pd.read_csv('games.csv')
games_details_df = pd.read_csv('games_details.csv')
players_df = pd.read_csv('players.csv')
ranking_df = pd.read_csv('ranking.csv')
teams_df = pd.read_csv('teams.csv')

C:\Users\fergb\AppData\Local\Temp\ipykernel_35100\2321718875.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details_df = pd.read_csv('games_details.csv')


### Preparing the Data

In [392]:
games_details_df = games_details_df[games_details_df['COMMENT'].isna()]

In [393]:
def min_to_float(min):
    if (type(min) == str):
        if (':' in min):
            if ("." in min):
                split = min.split('.')
                min = ":".join([split[0], split[1][-2:]])
            mins, secs = map(int, min.split(':'))
            return (mins + secs) / 60.0
        else:
            return float(min)
    return min

games_details_df["MIN"] = games_details_df["MIN"].apply(min_to_float)

In [394]:
game_dropped_columns = ["GAME_DATE_EST", "GAME_ID" , "GAME_STATUS_TEXT", "HOME_TEAM_ID", "VISITOR_TEAM_ID"]
details_dropped_columns = ["GAME_ID","TEAM_ID","TEAM_ABBREVIATION","TEAM_CITY","PLAYER_ID","PLAYER_NAME","NICKNAME","START_POSITION", 'COMMENT']

### Prediction Functionality

In [395]:
def get_team_id(name):
    return teams_df[teams_df['NICKNAME'] == name]['TEAM_ID'].values[0]

In [396]:
def predict_classifer(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X,              
                                                    y,              
                                                    test_size=0.3,  
                                                    random_state=7,
                                                    stratify=y
                                                   )
    rf = RandomForestClassifier(random_state=7)
    rf.fit(X_train, y_train)

    return rf.predict(X_test)

In [397]:
def predict_regressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X,              
                                                    y,              
                                                    test_size=0.3,  
                                                    random_state=7,
                                                
                                                   )
    rf = RandomForestRegressor(random_state=7)
    rf.fit(X_train, y_train)

    return rf.predict(X_test)

In [398]:
def predict_winner(home, away):
    X = games_df[(games_df["HOME_TEAM_ID"] == get_team_id(home)) & (games_df['VISITOR_TEAM_ID'] == get_team_id(away))]
    X = X.drop(columns=game_dropped_columns)
    y = X['HOME_TEAM_WINS']
    X = X.drop(columns='HOME_TEAM_WINS')
    
    predictions = predict_classifer(X, y)

    result = np.bincount(predictions).argmax()
    
    if (result == 1):
        return True
    else:
        return False

In [399]:
def predict_score(home, away):
    X = games_df[(games_df["HOME_TEAM_ID"] == get_team_id(home)) & (games_df['VISITOR_TEAM_ID'] == get_team_id(away))]
    X = X.drop(columns=game_dropped_columns)
    y = X[['PTS_home', 'PTS_away']]
    X = X.drop(columns=['PTS_home', 'PTS_away'])

    predictions = predict_regressor(X, y)

    home_scores = predictions[:, 0]
    away_scores = predictions[:, 1]

    return (math.floor(np.mean(home_scores)), math.floor(np.mean(away_scores)))

In [400]:
def predict_player_stat_against_team(player, team, stat):
    games_played_by_player = games_details_df[games_details_df['PLAYER_NAME'] == player]
    game_ids = games_played_by_player['GAME_ID'].tolist()
    
    games_ids_against_team = []
    for game_id in game_ids:
        game = games_df[games_df['GAME_ID'] == game_id]
        if (game["HOME_TEAM_ID"].values[0] == get_team_id(team) or game["VISITOR_TEAM_ID"].values[0] == get_team_id(team)):
            games_ids_against_team.append(game_id)

    games_against_team = games_played_by_player[games_played_by_player['GAME_ID'].isin(games_ids_against_team)]
    X = games_against_team.drop(columns=details_dropped_columns)
    y = X[stat]
    X = X.drop(columns=stat)

    predictions = predict_regressor(X, y)

    return math.floor(np.mean(predictions))

### Betting Functionality

In [401]:
def betting_number(pred, bet, is_betting_over):
     return (is_betting_over and bet < pred) or (not is_betting_over and bet > pred)

In [402]:
def make_win_bet(home, away, is_home_winning):
    pred = predict_winner(home, away)
    return is_home_winning == pred

In [411]:
def make_score_bet(home, away, bet, is_betting_home, is_betting_over):
    pred = predict_score(home, away)
    if (is_betting_home):
        return betting_number(pred[0], bet, is_betting_over), pred[0]
    else:
        return betting_number(pred[1], bet, is_betting_over), pred[1]

In [415]:
def make_player_stat_bet(player, stat, bet, opponent, is_beating_over):
    pred = predict_player_stat_against_team(player, opponent, stat)
    return betting_number(pred, bet, is_beating_over), pred

In [ ]:
win_iface = gr.Interface(
    fn=make_win_bet,
    inputs=[
        gr.Textbox("", label="Home Team"),
        gr.Textbox("", label="Away Team"),
        gr.Checkbox(label="Is Home Winning?")
    ],
    outputs=gr.Textbox(label='Correct'),
    theme="compact",
    title="Winning Bet Predictor"
)

win_iface.launch()

In [412]:
score_iface = gr.Interface(
    fn=make_score_bet,
    inputs=[
        gr.Textbox("", label="Home Team"),
        gr.Textbox("", label="Away Team"),
        gr.Number(label='Bet'),
        gr.Checkbox(label="Betting home team?"),
        gr.Checkbox(label='Betting Over?')
    ],
    outputs=[
        gr.Textbox(label='Correct'),
        gr.Textbox(label='Pred')
    ],
    theme="compact",
    title="Score Bet Predictor"
)

score_iface.launch()

C:\Users\fergb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\gradio\blocks.py:528: UserWarning: Cannot load compact. Caught Exception: The space compact does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


In [416]:
stat_iface = gr.Interface(
    fn=make_player_stat_bet,
    inputs=[
        gr.Textbox("", label="Player"),
        gr.Textbox("", label='Stat'),
        gr.Number(label='Bet'),
        gr.Textbox("", 'Opponent'),
        gr.Checkbox(label='Betting Over?')
    ],
    outputs=[
        gr.Textbox(label='Correct'),
        gr.Textbox(label='Pred')
    ],
    theme="compact",
    title="Stat Bet Predictor"
)

stat_iface.launch()

C:\Users\fergb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\gradio\blocks.py:528: UserWarning: Cannot load compact. Caught Exception: The space compact does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.
